In [ ]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F
from sentence_transformers import util, losses
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
fine_tuning_file_dir = './data/computers_train/computers_train_large.json.gz'
ft_valid_file_dir = './data/computers_valid/computers_valid_large.csv'
n_epochs = 1

In [ ]:
# finetuning bert model
large_df = pd.read_json(fine_tuning_file_dir, compression='gzip', lines=True)
large_df = large_df.drop(columns=['id_left', 'id_right', 'cluster_id_left', 'cluster_id_right', 'identifiers_left', 'identifiers_right'])
valid_ids = pd.read_csv(ft_valid_file_dir)
ft_train_df = large_df.loc[~large_df['pair_id'].isin(valid_ids['pair_id'])]

In [ ]:

titles_left = list(ft_train_df['title_left'])
titles_right = list(ft_train_df['title_right'])

descriptions_left = list(ft_train_df['description_left'])
descriptions_right = list(ft_train_df['description_right'])


In [ ]:
titles_left

In [ ]:
train_samples = []
for title_left, title_right in zip(titles_left, titles_right):
    train_samples.append(InputExample(
        texts=[title_left, title_right]
    ))

train_dataloader = DataLoader(
    train_samples, batch_size=8
)

title_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
loss = losses.MultipleNegativesRankingLoss(title_model)
warmup_steps = int(len(train_dataloader) * n_epochs * 0.1)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(train_samples, name='sts-dev')

In [ ]:
title_model.fit(
    train_objectives=[(train_dataloader, loss)],
    evaluator=evaluator,
    epochs=n_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path='./title_model_finetuned')

In [ ]:
train_samples = []
for desc_left, desc_right in zip(descriptions_left, descriptions_right):
    if desc_left != None and desc_right != None:
        train_samples.append(InputExample(
            texts=[desc_left, desc_right]
        ))

train_dataloader = DataLoader(
    train_samples, batch_size=8
)

desc_model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")
loss = losses.MultipleNegativesRankingLoss(desc_model)
warmup_steps = int(len(train_dataloader) * n_epochs * 0.1)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(train_samples, name='sts-dev')